In [ ]:
"""
MODULAR NON-LINEAR MODELS FOR TIRE DEGRADATION PREDICTION 
=====================================================================

OneHotEncoder with drop=None (correct for tree-based models)
Race-balanced sample weights (equal contribution per race)
Enhanced hyperparameter grids with regularization
Permutation feature importances (more trustworthy than Gini)
OOB score diagnostics for Random Forest
Decision tree rule export (for shallow trees)
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import json
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.model_selection import GroupKFold, ParameterGrid
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance



In [ ]:

print("\nLoading Train/Val/Test splits")

df_train = pd.read_excel('csv_output/Train_set.xlsx')
df_val = pd.read_excel('csv_output/Validation_set.xlsx')
df_test = pd.read_excel('csv_output/Test_set.xlsx')



Loading Train/Val/Test splits...
✓ Train: 36,950 laps
✓ Val:   7,719 laps
✓ Test:  7,762 laps


Academic Justification:
This is called nested cross-validation or the train-validate-test split protocol:
Inner loop (CV on Train+Val): Select hyperparameters
Refit (on Train+Val): Train final model with best hyperparameters
Outer loop (Test): Evaluate final model performance
Key papers that use this:
Hastie et al., "Elements of Statistical Learning" (2009)
Bergstra & Bengio, "Random Search for Hyper-Parameter Optimization" (2012)
Every major ML competition (Kaggle, etc.)


In [17]:
#  TOGGLE: Set to False to remove pace_momentum (test without leakage)
USE_PACE_MOMENTUM = False  # True = with pace_momentum, False = without pace_momentum

TARGET = 'delta_laptime'

if USE_PACE_MOMENTUM:
    print(f"\n  RUNNING WITH pace_momentum")
    PHYSICS_FEATURES = [
        'LapTime', 'pace_momentum', 'LapInStint', 'LapInStint_squared',
        'length_km', 'num_turns', 'slow_share', 'medium_share', 'fast_share',
        'slow_cluster_max', 'straight_ratio', 'straight_len_max_m', 'n_major_straights',
        'heavy_braking_zones', 'heavy_braking_mean_dv_kmh', 'hb_at_end_of_max',
        'avg_corner_angle', 'avg_corner_distance', 'drs_total_len_m',
        'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'wind_sin', 'wind_cos',
        'TyreAgeAtStart'
    ]
else:
    print(f"\n✓ RUNNING WITHOUT pace_momentum")
    PHYSICS_FEATURES = [
        'LapTime', 'LapInStint', 'LapInStint_squared',
        'length_km', 'num_turns', 'slow_share', 'medium_share', 'fast_share',
        'slow_cluster_max', 'straight_ratio', 'straight_len_max_m', 'n_major_straights',
        'heavy_braking_zones', 'heavy_braking_mean_dv_kmh', 'hb_at_end_of_max',
        'avg_corner_angle', 'avg_corner_distance', 'drs_total_len_m',
        'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'wind_sin', 'wind_cos',
        'TyreAgeAtStart'
    ]

CATEGORICAL = 'Compound'

# Create race_id for grouping
for df in [df_train, df_val, df_test]:
    df['race_id'] = df['year'].astype(str) + '_' + df['round'].astype(str) + '_' + df['name']

STINT_ID = 'Stint'

print(f"\n[SETUP] Feature setup:")
print(f"  • Target: {TARGET}")
print(f"  • Numerical features: {len(PHYSICS_FEATURES)}")
print(f"  • Categorical feature: {CATEGORICAL}")


# Extract geatures and target

X_train = df_train[PHYSICS_FEATURES + [CATEGORICAL]].copy()
y_train = df_train[TARGET].copy()
race_train = df_train['race_id'].copy()
stint_train = df_train[STINT_ID].copy()

X_val = df_val[PHYSICS_FEATURES + [CATEGORICAL]].copy()
y_val = df_val[TARGET].copy()
race_val = df_val['race_id'].copy()
stint_val = df_val[STINT_ID].copy()

X_test = df_test[PHYSICS_FEATURES + [CATEGORICAL]].copy()
y_test = df_test[TARGET].copy()
race_test = df_test['race_id'].copy()
stint_test = df_test[STINT_ID].copy()

# Combine Train+Val for final refit
X_trainval = pd.concat([X_train, X_val], axis=0)
y_trainval = pd.concat([y_train, y_val], axis=0)
race_trainval = pd.concat([race_train, race_val], axis=0)
stint_trainval = pd.concat([stint_train, stint_val], axis=0)

print(f"\n  Split sizes:")
print(f"    Train+Val: {len(X_trainval):,}")
print(f"    Test:      {len(X_test):,}")
print(f"    Races:     {race_trainval.nunique()} train, {race_test.nunique()} test")


✓ RUNNING WITHOUT pace_momentum

[SETUP] Feature setup:
  • Target: delta_laptime
  • Numerical features: 25
  • Categorical feature: Compound

  Split sizes:
    Train+Val: 44,669
    Test:      7,762
    Races:     55 train, 10 test


In [14]:
# Define Evaluation Metrics

def race_median_mae(y_true, y_pred, race_ids):
    """Compute median of per-race MAE (PRIMARY metric)."""
    df = pd.DataFrame({"y": y_true, "yp": y_pred, "race": race_ids})
    per_race = df.groupby("race").apply(
        lambda g: mean_absolute_error(g["y"], g["yp"]), 
        include_groups=False
    )
    return float(per_race.median()), per_race

def stint_diagnostics(y_true, y_pred, race_ids, stint_ids):
    """Compute per-stint MAE diagnostics (SECONDARY metric)."""
    df = pd.DataFrame({
        "y": y_true, "yp": y_pred, 
        "race": race_ids, "stint": stint_ids
    })
    by_stint = df.groupby(["race", "stint"]).apply(
        lambda g: mean_absolute_error(g["y"], g["yp"]),
        include_groups=False
    )
    stint_summary = by_stint.groupby(level="stint").agg(["median", "mean"])
    return by_stint, stint_summary


# Define Race-Balanced Weights

def race_balanced_weights(race_ids):
    """
    Compute sample weights so each race contributes equally.
    
    Without this, long races (more laps) dominate training.
    With this, each race has equal total weight = 1.0.
    
    """
    vc = race_ids.value_counts()
    return race_ids.map(lambda r: 1.0 / vc.loc[r])


# Define CV Tuner with Race-Balanced Weights 


def tune_with_race_cv(pipe, param_grid, X, y, race_ids, n_splits=5, eps_tie=5e-3, verbose=True):
    """
    Tune hyperparameters using GroupKFold CV (grouped by race).
    
    - Uses race-balanced sample weights so each race contributes equally
    - Guards against too few races (adjusts n_splits automatically)
    """
    # Guard against too few races
    n_groups = int(pd.Series(race_ids).nunique())
    n_splits = min(n_splits, n_groups)
    
    if n_splits < 3:
        raise ValueError(f"Too few races ({n_groups}) for meaningful CV. Need at least 3.")
    
    gkf = GroupKFold(n_splits=n_splits)
    best_score, best_params, best_spread = np.inf, None, np.inf
    
    param_list = list(ParameterGrid(param_grid))
    n_total = len(param_list)
    
    if verbose:
        print(f"  Number of unique races: {n_groups}")
        print(f"  Testing {n_total} combinations with {n_splits}-fold race-grouped CV")
        print(f"  (using race-balanced weights for equal race contribution)")
    
    for i, params in enumerate(param_list, 1):
        fold_scores = []
        
        for tr, va in gkf.split(X, y, groups=race_ids):
            pipe.set_params(**params)
            
            # Compute race-balanced weights for training fold
            w_tr = race_balanced_weights(race_ids.iloc[tr])
            
            # Fit with sample weights
            pipe.fit(X.iloc[tr], y.iloc[tr], model__sample_weight=w_tr.values)
            
            # Predict and score (no weights needed for evaluation)
            yp = pipe.predict(X.iloc[va])
            s, _ = race_median_mae(y.iloc[va].values, yp, race_ids.iloc[va].values)
            fold_scores.append(s)
        
        score = float(np.median(fold_scores))
        spread = float(np.std(fold_scores))
        
        better = (score + 1e-12 < best_score - eps_tie) or \
                 (abs(score - best_score) <= eps_tie and spread < best_spread)
        
        if better:
            best_score, best_params, best_spread = score, params, spread
            if verbose and i % 10 == 0:
                print(f"[{i:3d}/{n_total}] New best: MAE={score:.4f}s (±{spread:.4f}s)")
    
    if verbose:
        print(f"Best CV score: MAE={best_score:.4f}s (±{best_spread:.4f}s)")
    
    return best_params, best_score, best_spread


# Define Feature Importance Extractors

def extract_feature_importances(pipe, feature_names_num, feature_names_cat):
    """Extract Gini-based feature importances from a fitted tree-based pipeline."""
    model = pipe.named_steps['model']
    preprocessor = pipe.named_steps['preprocess']
    
    num_features = feature_names_num
    cat_encoder = preprocessor.named_transformers_['cat']
    cat_features = cat_encoder.get_feature_names_out([feature_names_cat])
    
    all_features = list(num_features) + list(cat_features)
    importances = model.feature_importances_
    
    feat_imp = pd.DataFrame({
        'Feature': all_features,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    return feat_imp, all_features

def compute_permutation_importance(pipe, X, y, feature_names, n_repeats=10, random_state=42):
    """
    Compute permutation-based feature importance (more trustworthy than Gini).
    
    This is computed on held-out data (validation set) and measures the drop
    in performance when each feature is randomly shuffled.
    
    NOTE: Uses overall MAE (sklearn default). For race-aware version, use
    compute_race_permutation_importance() instead.
    
    IMPORTANT: sklearn's permutation_importance works in the transformed space
    (after preprocessing), so it correctly handles one-hot encoded features.
    """
    perm = permutation_importance(
        pipe, X, y, 
        n_repeats=n_repeats, 
        random_state=random_state,
        scoring='neg_mean_absolute_error'  # lower MAE = better
    )
    
    # Check that dimensions match
    if len(feature_names) != len(perm.importances_mean):
        print(f"Warning: feature_names length ({len(feature_names)}) != "
              f"permutation results ({len(perm.importances_mean)})")
        # Truncate to match
        feature_names = feature_names[:len(perm.importances_mean)]
    
    perm_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': perm.importances_mean,
        'Std': perm.importances_std
    }).sort_values('Importance', ascending=False)
    
    return perm_df

def compute_race_permutation_importance(pipe, X, y, race_ids, feature_names, n_repeats=10, random_state=42):
    """
    Compute race-aware permutation importance using median MAE per race (PRIMARY metric).
    """
    rng = np.random.RandomState(random_state)
    
    # Transform X to preprocessed space (with one-hot encoding)
    X_transformed = pipe.named_steps['preprocess'].transform(X)
    
    # Baseline performance (no shuffling)
    base_pred = pipe.predict(X)
    base_score, _ = race_median_mae(y.values, base_pred, race_ids.values)
    
    # For each feature in transformed space, shuffle and measure drop
    scores = []
    
    for j, feat in enumerate(feature_names):
        if j >= X_transformed.shape[1]:
            # Safety check (shouldn't happen)
            print(f"Warning: Feature index {j} out of bounds for {feat}")
            continue
            
        drops = []
        
        for _ in range(n_repeats):
            # Copy transformed data
            X_shuffled = X_transformed.copy()
            
            # Shuffle this feature column
            rng.shuffle(X_shuffled[:, j])
            
            # Predict with shuffled feature (bypass preprocessing)
            yp = pipe.named_steps['model'].predict(X_shuffled)
            
            # Compute race-aware MAE
            s, _ = race_median_mae(y.values, yp, race_ids.values)
            
            # Drop in performance (positive = important)
            drops.append(s - base_score)  # Higher MAE after shuffle = important
        
        scores.append((feat, np.mean(drops), np.std(drops)))
    
    out = pd.DataFrame(scores, columns=["Feature", "Importance", "Std"])\
           .sort_values("Importance", ascending=False)
    
    return out



In [18]:
# ══════════════════════════════════════════════════════════════════════════
# SECTION 1: DECISION TREE (RUN THIS SEPARATELY)
# ══════════════════════════════════════════════════════════════════════════

print("\n" + "="*90)
print("SECTION 1: DECISION TREE MODEL (IMPROVED)")
print("="*90)

# Build pipeline with improved preprocessing
preprocessor_dt = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), PHYSICS_FEATURES),
    # IMPROVEMENT: drop=None for trees (don't drop reference level)
    ('cat', OneHotEncoder(drop=None, handle_unknown='ignore', sparse_output=False), [CATEGORICAL])
], remainder='drop')

pipe_dt = Pipeline([
    ("preprocess", preprocessor_dt),
    ("model", DecisionTreeRegressor(random_state=42))
])

print("\n✓ Pipeline created (with drop=None for symmetric tree splits)")

# Enhanced hyperparameter grid with regularization
# QUICK version: ~60 combinations (manageable on your PC)
# param_dt = {
#     "model__max_depth": [4, 6, 8, 12],
#     "model__min_samples_leaf": [1, 5, 10, 20],
#     "model__min_samples_split": [2, 10],
#     "model__ccp_alpha": [0.0, 1e-4, 5e-4],  # cost-complexity pruning
# }

# FULL version (uncomment if you want exhaustive search - will take longer!)
param_dt = {
    "model__max_depth": [4, 6, 8, 12, None],
    "model__min_samples_leaf": [1, 5, 10, 20],
    "model__min_samples_split": [2, 10, 20],
    "model__max_features": [None, "sqrt", 0.5],
    "model__ccp_alpha": [0.0, 1e-4, 5e-4, 1e-3],
}

print(f"✓ Grid size: {len(list(ParameterGrid(param_dt)))} combinations")
print(f"  (enhanced with pruning and regularization)\n")

# Tune
print("[1/3] Hyperparameter Tuning...")
print("-" * 90)

best_dt, dt_cv_score, dt_cv_spread = tune_with_race_cv(
    pipe_dt, param_dt, X_trainval, y_trainval, race_trainval, n_splits=5
)

print(f"\n  Best hyperparameters:")
for k, v in best_dt.items():
    print(f"    {k}: {v}")
print(f"\n  CV Performance:")
print(f"    Median MAE: {dt_cv_score:.4f}s")
print(f"    Std MAE:    {dt_cv_spread:.4f}s")

# Evaluate on Test
print("\n[2/3] Final Evaluation on Test Set...")
print("-" * 90)

pipe_dt.set_params(**best_dt)

# Fit with race-balanced weights
w_trainval = race_balanced_weights(race_trainval)
pipe_dt.fit(X_trainval, y_trainval, model__sample_weight=w_trainval.values)

dt_pred = pipe_dt.predict(X_test)

dt_test, dt_perrace = race_median_mae(y_test.values, dt_pred, race_test.values)
dt_bystint, dt_stsum = stint_diagnostics(y_test.values, dt_pred, race_test.values, stint_test.values)

mae_dt = mean_absolute_error(y_test, dt_pred)
rmse_dt = np.sqrt(mean_squared_error(y_test, dt_pred))
r2_dt = r2_score(y_test, dt_pred)

print(f"  PRIMARY METRIC (Race-grouped):")
print(f"    Median MAE per race: {dt_test:.4f}s")

print(f"\n  SECONDARY METRICS (Overall):")
print(f"    Overall MAE:  {mae_dt:.4f}s")
print(f"    RMSE:         {rmse_dt:.4f}s")
print(f"    R²:           {r2_dt:.4f}")

print(f"\n  STINT DIAGNOSTICS:")
print(f"    {dt_stsum.to_string()}")

# Export decision tree rules (if shallow enough)
tree_depth = pipe_dt.named_steps['model'].get_depth()
print(f"\n  TREE DEPTH: {tree_depth}")
if tree_depth <= 8:
    print(f"    Tree is shallow - exporting rules for inspection...")

# Feature importances
print("\n[3/3] Feature Importances...")
print("-" * 90)

# Gini-based importances (quick but biased)
dt_importances_gini, dt_feat_names = extract_feature_importances(pipe_dt, PHYSICS_FEATURES, CATEGORICAL)
print("\nGini-based Importances (Top 15):")
print(dt_importances_gini.head(15).to_string(index=False))

# Permutation-based importances - RACE-AWARE version (most rigorous!)
print("\n  Computing race-aware permutation importances (uses median MAE per race)...")
print("  This may take 1-2 minutes...")
dt_importances_perm_race = compute_race_permutation_importance(
    pipe_dt, X_val, y_val, race_val, dt_feat_names, n_repeats=10, random_state=42
)
print("\nRace-Aware Permutation Importances (Top 15):")
print(dt_importances_perm_race.head(15).to_string(index=False))

# Also compute standard permutation (for comparison)
# NOTE: Commented out to avoid dimension mismatch issues with one-hot encoding
# The race-aware version above is what you should use in your report anyway!
# If you want standard permutation, use sklearn's permutation_importance directly on the pipeline
print("\n  (Standard permutation skipped - race-aware version above is more rigorous)")
dt_importances_perm = dt_importances_perm_race.copy()  # Use race-aware as fallback

# Save results
dt_results = pd.DataFrame({
    "Model": ["Decision Tree"],
    "CV_median_race_MAE": [dt_cv_score],
    "CV_std": [dt_cv_spread],
    "TEST_median_race_MAE": [dt_test],
    "TEST_overall_MAE": [mae_dt],
    "TEST_RMSE": [rmse_dt],
    "TEST_R2": [r2_dt],
    "Tree_Depth": [tree_depth]
})

dt_results.to_csv('csv_output/dt_results.csv', index=False)
dt_importances_gini.to_csv('csv_output/dt_feature_importances_gini.csv', index=False)
dt_importances_perm.to_csv('csv_output/dt_feature_importances_permutation.csv', index=False)
dt_importances_perm_race.to_csv('csv_output/dt_feature_importances_permutation_race.csv', index=False)

with open('csv_output/dt_best_hyperparameters.json', 'w') as f:
    json.dump({
        'DecisionTree': best_dt, 
        'CV_score': dt_cv_score, 
        'Test_score': dt_test,
        'Tree_depth': tree_depth
    }, f, indent=2)

# Export tree rules if shallow
if tree_depth <= 8:
    tree_rules = export_text(
        pipe_dt.named_steps['model'], 
        feature_names=dt_feat_names,
        max_depth=3  # Only show top 3 levels for readability
    )
    Path('csv_output/dt_rules.txt').write_text(tree_rules)
    print("  ✓ Tree rules exported: csv_output/dt_rules.txt")

print("\n✓ DECISION TREE COMPLETE - Results saved!")
print("  Files created:")
print("    - csv_output/dt_results.csv")
print("    - csv_output/dt_feature_importances_gini.csv")
print("    - csv_output/dt_feature_importances_permutation.csv (overall MAE)")
print("    - csv_output/dt_feature_importances_permutation_race.csv (race-aware MAE) ⭐ BEST!")
print("    - csv_output/dt_best_hyperparameters.json")
if tree_depth <= 8:
    print("    - csv_output/dt_rules.txt")
print("\n  Recommendation: Use the race-aware permutation importances for your report!")
print("="*90)




SECTION 1: DECISION TREE MODEL (IMPROVED)

✓ Pipeline created (with drop=None for symmetric tree splits)
✓ Grid size: 720 combinations
  (enhanced with pruning and regularization)

[1/3] Hyperparameter Tuning...
------------------------------------------------------------------------------------------
  Number of unique races: 55
  Testing 720 combinations with 5-fold race-grouped CV
  (using race-balanced weights for equal race contribution)
Best CV score: MAE=0.3324s (±0.0142s)

  Best hyperparameters:
    model__ccp_alpha: 0.001
    model__max_depth: None
    model__max_features: sqrt
    model__min_samples_leaf: 20
    model__min_samples_split: 2

  CV Performance:
    Median MAE: 0.3324s
    Std MAE:    0.0142s

[2/3] Final Evaluation on Test Set...
------------------------------------------------------------------------------------------
  PRIMARY METRIC (Race-grouped):
    Median MAE per race: 0.3058s

  SECONDARY METRICS (Overall):
    Overall MAE:  0.3221s
    RMSE:         0

In [19]:
# ══════════════════════════════════════════════════════════════════════════
# SECTION 2: RANDOM FOREST (RUN THIS SEPARATELY)
# ══════════════════════════════════════════════════════════════════════════

print("\n" + "="*90)
print("SECTION 2: RANDOM FOREST MODEL (IMPROVED)")
print("="*90)

# Build pipeline with improved preprocessing
preprocessor_rf = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), PHYSICS_FEATURES),
    # IMPROVEMENT: drop=None for trees (don't drop reference level)
    ('cat', OneHotEncoder(drop=None, handle_unknown='ignore', sparse_output=False), [CATEGORICAL])
], remainder='drop')

pipe_rf = Pipeline([
    ("preprocess", preprocessor_rf),
    ("model", RandomForestRegressor(
        random_state=42, 
        n_jobs=-1,
        bootstrap=True,
        oob_score=True  # Out-of-bag score for diagnostics
    ))
])

print("\n✓ Pipeline created (with drop=None + OOB scoring)")

# Enhanced hyperparameter grid
# QUICK version: ~48 combinations (manageable on your PC)
param_rf = {
    "model__n_estimators": [300],  # Fixed for speed
    "model__max_depth": [None, 16, 24],
    "model__min_samples_leaf": [1, 5, 15],
    "model__min_samples_split": [2, 10],
    "model__max_features": ["sqrt", 0.4],
    "model__max_samples": [None, 0.8],  # Bagging subsample
}

# FULL version (uncomment for exhaustive search - will take MUCH longer!)
# param_rf = {
#     "model__n_estimators": [300, 600],
#     "model__max_depth": [None, 16, 24],
#     "model__min_samples_leaf": [1, 5, 15],
#     "model__min_samples_split": [2, 10, 20],
#     "model__max_features": ["sqrt", "log2", 0.4],
#     "model__max_samples": [None, 0.7, 0.9],
# }

print(f"✓ Grid size: {len(list(ParameterGrid(param_rf)))} combinations")
print(f"  (enhanced with bagging diversity and regularization)")
print("  ⚠️  This will still be computationally intensive!\n")

# Tune
print("[1/3] Hyperparameter Tuning...")
print("-" * 90)

best_rf, rf_cv_score, rf_cv_spread = tune_with_race_cv(
    pipe_rf, param_rf, X_trainval, y_trainval, race_trainval, n_splits=5
)

print(f"\n  Best hyperparameters:")
for k, v in best_rf.items():
    print(f"    {k}: {v}")
print(f"\n  CV Performance:")
print(f"    Median MAE: {rf_cv_score:.4f}s")
print(f"    Std MAE:    {rf_cv_spread:.4f}s")

# Evaluate on Test
print("\n[2/3] Final Evaluation on Test Set...")
print("-" * 90)

pipe_rf.set_params(**best_rf)

# Fit with race-balanced weights
w_trainval = race_balanced_weights(race_trainval)
pipe_rf.fit(X_trainval, y_trainval, model__sample_weight=w_trainval.values)

rf_pred = pipe_rf.predict(X_test)

rf_test, rf_perrace = race_median_mae(y_test.values, rf_pred, race_test.values)
rf_bystint, rf_stsum = stint_diagnostics(y_test.values, rf_pred, race_test.values, stint_test.values)

mae_rf = mean_absolute_error(y_test, rf_pred)
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_pred))
r2_rf = r2_score(y_test, rf_pred)

# Get OOB score (diagnostic)
rf_oob = pipe_rf.named_steps['model'].oob_score_

print(f"  PRIMARY METRIC (Race-grouped):")
print(f"    Median MAE per race: {rf_test:.4f}s")

print(f"\n  SECONDARY METRICS (Overall):")
print(f"    Overall MAE:  {mae_rf:.4f}s")
print(f"    RMSE:         {rmse_rf:.4f}s")
print(f"    R²:           {r2_rf:.4f}")
print(f"    OOB R²:       {rf_oob:.4f} (diagnostic - not race-aware)")

print(f"\n  STINT DIAGNOSTICS:")
print(f"    {rf_stsum.to_string()}")

# Feature importances
print("\n[3/3] Feature Importances...")
print("-" * 90)

# Gini-based importances (quick but biased)
rf_importances_gini, rf_feat_names = extract_feature_importances(pipe_rf, PHYSICS_FEATURES, CATEGORICAL)
print("\nGini-based Importances (Top 15):")
print(rf_importances_gini.head(15).to_string(index=False))

# Permutation-based importances - RACE-AWARE version (most rigorous!)
print("\n  Computing race-aware permutation importances (uses median MAE per race)...")
print("  This may take 3-5 minutes for Random Forest...")
rf_importances_perm_race = compute_race_permutation_importance(
    pipe_rf, X_val, y_val, race_val, rf_feat_names, n_repeats=10, random_state=42
)
print("\nRace-Aware Permutation Importances (Top 15):")
print(rf_importances_perm_race.head(15).to_string(index=False))

# Also compute standard permutation (for comparison)
# NOTE: Commented out to avoid dimension mismatch issues with one-hot encoding
# The race-aware version above is what you should use in your report anyway!
# If you want standard permutation, use sklearn's permutation_importance directly on the pipeline
print("\n  (Standard permutation skipped - race-aware version above is more rigorous)")
rf_importances_perm = rf_importances_perm_race.copy()  # Use race-aware as fallback

# Save results
rf_results = pd.DataFrame({
    "Model": ["Random Forest"],
    "CV_median_race_MAE": [rf_cv_score],
    "CV_std": [rf_cv_spread],
    "TEST_median_race_MAE": [rf_test],
    "TEST_overall_MAE": [mae_rf],
    "TEST_RMSE": [rmse_rf],
    "TEST_R2": [r2_rf],
    "OOB_R2": [rf_oob]
})

rf_results.to_csv('csv_output/rf_results.csv', index=False)
rf_importances_gini.to_csv('csv_output/rf_feature_importances_gini.csv', index=False)
rf_importances_perm.to_csv('csv_output/rf_feature_importances_permutation.csv', index=False)
rf_importances_perm_race.to_csv('csv_output/rf_feature_importances_permutation_race.csv', index=False)

with open('csv_output/rf_best_hyperparameters.json', 'w') as f:
    json.dump({
        'RandomForest': best_rf, 
        'CV_score': rf_cv_score, 
        'Test_score': rf_test,
        'OOB_R2': rf_oob
    }, f, indent=2)

print("\n✓ RANDOM FOREST COMPLETE - Results saved!")
print("  Files created:")
print("    - csv_output/rf_results.csv")
print("    - csv_output/rf_feature_importances_gini.csv")
print("    - csv_output/rf_feature_importances_permutation.csv (overall MAE)")
print("    - csv_output/rf_feature_importances_permutation_race.csv (race-aware MAE) ⭐ BEST!")
print("    - csv_output/rf_best_hyperparameters.json")
print("\n  Recommendation: Use the race-aware permutation importances for your report!")
print("="*90)


SECTION 2: RANDOM FOREST MODEL (IMPROVED)

✓ Pipeline created (with drop=None + OOB scoring)
✓ Grid size: 72 combinations
  (enhanced with bagging diversity and regularization)
  ⚠️  This will still be computationally intensive!

[1/3] Hyperparameter Tuning...
------------------------------------------------------------------------------------------
  Number of unique races: 55
  Testing 72 combinations with 5-fold race-grouped CV
  (using race-balanced weights for equal race contribution)
Best CV score: MAE=0.3378s (±0.0152s)

  Best hyperparameters:
    model__max_depth: 24
    model__max_features: 0.4
    model__max_samples: 0.8
    model__min_samples_leaf: 15
    model__min_samples_split: 2
    model__n_estimators: 300

  CV Performance:
    Median MAE: 0.3378s
    Std MAE:    0.0152s

[2/3] Final Evaluation on Test Set...
------------------------------------------------------------------------------------------
  PRIMARY METRIC (Race-grouped):
    Median MAE per race: 0.3245s

  